# Оценка баланса реактивной мощности

## Проверка по условию выполнения БРМ в сети и определение приведенных к шинам высшего напряжения нагрузок подстанции 

In [2]:
from IPython.display import Math, display
import numpy as np
%store -r Q_nb_i
%store -r S_nb_i
%store -r Lij

In [3]:
#                 K1      K2    1-3   2-3   3-4  4-5
P_1c = np.array([125.3, 119.7, 55.3, 49.7, 37.5, 17.5])
L_kolca = np.array([Lij[0, 1], Lij[0, 2], Lij[1, 3], Lij[2, 3], Lij[3, 4], Lij[4, 5]])
mask_p1c = [False, False, False, False, True, True]
L_kolca

array([41.8, 49.2, 49.2, 41.8, 25.9, 25.9])

In [4]:
k_odn_q = 0.98
mask_ps = [False, False, False, True, True]
Q_treb_1 = np.around(k_odn_q * np.sum(Q_nb_i), 1)
print(f'Q_treb_1 = {Q_treb_1}')

Q_treb_2 = 0.08 * np.sum(np.where(mask_ps, S_nb_i * 2, S_nb_i))
print(f'Q_treb_2 = {Q_treb_2}')

q_c0_220 = 0.14
q_c0_110 = 0.036
i = 0
Q_c_kolco = np.zeros_like(L_kolca)
for i in range(len(L_kolca)):
    if mask_p1c[i]:
        Q_c_kolco[i] = np.around(2 * q_c0_110 * L_kolca[i], 2)
    else:
        Q_c_kolco[i] = np.around(q_c0_220 * L_kolca[i], 2)
print(f'Q_c_kolco = {Q_c_kolco}')

Q_treb_1 = 93.5
Q_treb_2 = 27.48
Q_c_kolco = [5.85 6.89 6.89 5.85 1.86 1.86]


In [5]:
P_nat_220 = 130
P_nat_110 = 30
i = 0
delta_Q_l = np.zeros_like(Q_c_kolco)
for i in range(len(Q_c_kolco)):
    if mask_p1c[i] and P_1c[i] > P_nat_110:
        delta_Q_l[i] = np.around((P_1c[i] / P_nat_110)**2 * Q_c_kolco[i], 2)
    elif mask_p1c[i] and P_1c[i] <= P_nat_110:
        delta_Q_l[i] = Q_c_kolco[i]
    else:
        delta_Q_l[i] = np.around((P_1c[i] / P_nat_220)**2 * Q_c_kolco[i], 2)

Q_treb_3 = np.around(np.sum(delta_Q_l - Q_c_kolco), 1)

Q_treb_sum = np.around(Q_treb_1 + Q_treb_2 + Q_treb_3, 1)

print(f'Q_c_kolco = {Q_c_kolco}')
print(f'Q_treb_3 = {Q_treb_3} \nQ_treb_sum = {Q_treb_sum}')

Q_c_kolco = [5.85 6.89 6.89 5.85 1.86 1.86]
Q_treb_3 = -11.0 
Q_treb_sum = 110.0


In [6]:
print(f'L = {L_kolca} \nP_1c = {P_1c} \nQ_c = {Q_c_kolco} \ndelta_Q_l = {delta_Q_l}')
print(np.sum(delta_Q_l) - delta_Q_l[5])

L = [41.8 49.2 49.2 41.8 25.9 25.9] 
P_1c = [125.3 119.7  55.3  49.7  37.5  17.5] 
Q_c = [5.85 6.89 6.89 5.85 1.86 1.86] 
delta_Q_l = [5.43 5.84 1.25 0.86 2.91 1.86]
16.29


## Расстановка КУ

In [7]:
%store -r P_nb
%store -r tg_nb_i
q_rasp = 87
Q_dop_sum = Q_treb_sum - q_rasp
tg_phi_b = np.around((np.sum(Q_nb_i) - Q_dop_sum) / np.sum(P_nb), 3) # высчитываем для всех подстанций

print(tg_phi_b)
print(Q_dop_sum)
print(tg_nb_i)

0.296
23.0
[0.391 0.391 0.377 0.392 0.389]


In [8]:
Q_comp_dop = np.around(P_nb * (tg_nb_i - tg_phi_b), 2)
print("Q_comp_dop до округления вручную = ", Q_comp_dop)

Q_comp_dop = np.array([6.65, 6.65, 2.5, 3.9, 3.3]) # Вручную округляем так, чтобы сходилась сумма Qкуi (Q_comp_dop) и Qку_суммарное (Q_dop_sum) 
print("Q_comp_dop = ", Q_comp_dop)

N_bsk = np.around(Q_comp_dop / 1.2, 2)
N_bsk_treb = np.sum(Q_comp_dop) / 1.2

print("N_bsk до округления = ", N_bsk)

N_bsk_sum = 20

print("N_bsk_treb = ", N_bsk_treb)

N_bsk = np.array([6, 6, 2, 4, 2]) # уточняем количество БСК вручную

print("N_bsk_sum = ", N_bsk_sum)
print("N_bsk = ", N_bsk, "\nN_bsk = N_bsk_sum? ---", np.sum(N_bsk) == N_bsk_sum)

Q_comp_dop до округления вручную =  [6.65 6.65 2.43 3.84 3.26]
Q_comp_dop =  [6.65 6.65 2.5  3.9  3.3 ]
N_bsk до округления =  [5.54 5.54 2.08 3.25 2.75]
N_bsk_treb =  19.166666666666668
N_bsk_sum =  20
N_bsk =  [6 6 2 4 2] 
N_bsk = N_bsk_sum? --- True


In [9]:
Q_bsk_dop = N_bsk * 1.2
Q2_nb = np.around(Q_nb_i - Q_bsk_dop, 1) # здесь Q2_nb это Q''_нбi и далее двоечка будет означать апостроф
S2_nb = np.around(np.sqrt(P_nb**2 + Q2_nb**2), 1)
Q_priv = np.around(Q2_nb + 0.08 * S2_nb, 1)
S_priv = np.around(np.sqrt(P_nb**2 + Q_priv**2), 1)

Q_priv[3-1] = np.around(Q2_nb[3-1] + Q_priv[3] + Q_priv[4] + 0.08 * np.sqrt((P_nb[2] + P_nb[3] + P_nb[4])**2 + (Q2_nb[3-1] + Q_priv[3] + Q_priv[4])**2), 1)
S_priv[3-1] = np.around(np.sqrt((P_nb[2] + P_nb[3] + P_nb[4])**2 + Q_priv[2]**2), 1)

print("Q_bsk_dop = ", Q_bsk_dop, "\nQ''_nb = ", Q2_nb, "\nS''_nb = ", S2_nb, "\nQ_priv = ", Q_priv, "\nS_priv = ", S_priv)

Q_bsk_dop =  [7.2 7.2 2.4 4.8 2.4] 
Q''_nb =  [20.2 20.2  8.9 10.9 11.2] 
S''_nb =  [72.9 72.9 31.3 41.5 36.7] 
Q_priv =  [26.  26.  46.1 14.2 14.1] 
S_priv =  [ 74.7  74.7 114.7  42.4  37.7]


In [25]:
display(Math(f'''Q_{{прив3}} = Q_{{нб3}}^{{''}} + Q_{{прив4}} + Q_{{прив5}} + 0,08 \sqrt{{(P_{{нб3}} + P_{{нб4}} + P_{{прив5}})^2 + (Q_{{нб3}}^{{''}} + Q_{{прив4}} + Q_{{прив5}})^2}} = 
             {Q2_nb[3-1]} + {Q_priv[3]} + {Q_priv[4]} + 0,08 \sqrt{{({P_nb[2]} + {P_nb[3]} + {P_nb[4]})^2 + ({Q2_nb[3-1]} + {Q_priv[3]} + {Q_priv[4]})^2}} = {Q_priv[3-1]}'''))

<IPython.core.display.Math object>

In [27]:
display(Math(f'''S_{{прив3}} = \sqrt{{(P_{{нб3}} + P_{{нб4}} + P_{{нб5}})^2 + (Q_{{прив3}})^2}} = 
                \sqrt{{({P_nb[2]} + {P_nb[3]} + {P_nb[4]})^2 + ({Q_priv[2]})^2}} = {S_priv[3-1]}'''))

<IPython.core.display.Math object>

# Выбор сечений проводов линий электропередачи и их проверка по условиям технических ограничений

In [12]:
j_ek = 0.9
alpha_i = 1.05
#              K1   K2   1-3  2-3  3-4   4-5
Q = np.array([50.1, 48, 24.1, 22, 28.3, 14.1])
P = np.copy(P_1c)
P[mask_p1c] = P_1c[mask_p1c] * 2

i = 0
I_nb_5 = np.zeros_like(L_kolca)
for i in range(len(L_kolca)):
    if mask_p1c[i]:
        I_nb_5[i] = np.around(np.sqrt(P[i]**2 + Q[i]**2) / (np.sqrt(3) * 110 * 2) * 1000, 1)
    else:
        I_nb_5[i] = np.around(np.sqrt(P[i]**2 + Q[i]**2) / (np.sqrt(3) * 220 * 1) * 1000, 1)

I_r = alpha_i * I_nb_5
F_ek = np.around(I_r / j_ek, 1)

print(f'P = {P} \nQ = {Q} \nS = {np.around(np.sqrt(P**2 + Q**2), 1)}')
print("I_нб_(5) = ", I_nb_5, "\nI_р = ", I_r, "\nF_эк = ", F_ek)

P = [125.3 119.7  55.3  49.7  75.   35. ] 
Q = [50.1 48.  24.1 22.  28.3 14.1] 
S = [134.9 129.   60.3  54.4  80.2  37.7]
I_нб_(5) =  [354.1 338.4 158.3 142.6 210.4  99. ] 
I_р =  [371.805 355.32  166.215 149.73  220.92  103.95 ] 
F_эк =  [413.1 394.8 184.7 166.4 245.5 115.5]


## Проверка по условию длительно допустимого нагрева

In [13]:
k_t = 1.35
I_dl_dop = np.array([825, 825, 605, 605, 605, 390]) # эти значения зависят от марки провода и определяются по п. 1.3.29 ПУЭ
I_dl_dop_cor = I_dl_dop * k_t
print(I_dl_dop_cor)

[1113.75 1113.75  816.75  816.75  816.75  526.5 ]


## Оценка технической осуществимости схемы

In [14]:
R_0 = np.array([0.073, 0.073, 0.118, 0.118, 0.118, 0.244])
X_0 = np.array([0.42, 0.42, 0.435, 0.435, 0.405, 0.427])
B_0 = np.array([2.701, 2.701, 2.604, 2.604, 2.808, 2.658])

In [15]:
i = 0
R = np.zeros_like(L_kolca)
for i in range (len(L_kolca)):
    if mask_p1c[i]:
        R[i] = np.around(R_0[i] * L_kolca[i] / 2, 2)
    else:
        R[i] = np.around(R_0[i] * L_kolca[i], 2)

i = 0
X = np.zeros_like(L_kolca)
for i in range (len(L_kolca)):
    if mask_p1c[i]:
        X[i] = np.around(X_0[i] * L_kolca[i] / 2, 2)
    else:
        X[i] = np.around(X_0[i] * L_kolca[i], 2)
        
i = 0
B = np.zeros_like(L_kolca)
for i in range (len(L_kolca)):
    if mask_p1c[i]:
        B[i] = np.around(B_0[i] * L_kolca[i] * 2 / 2, 2)
    else:
        B[i] = np.around(B_0[i] * L_kolca[i] / 2, 2)

In [16]:
print(f'R = {R} \nX = {X} \nB/2 = {B} \nB = {B*2}') 

R = [3.05 3.59 5.81 4.93 1.53 3.16] 
X = [17.56 20.66 21.4  18.18  5.24  5.53] 
B/2 = [56.45 66.44 64.06 54.42 72.73 68.84] 
B = [112.9  132.88 128.12 108.84 145.46 137.68]


# Выбор трансформаторов и автотрансформаторов на подстанциях

In [17]:
k_av = 1.4
n_t = 2
S_t = np.around(S2_nb / (k_av * (n_t -1)), 1)
S_t = np.delete(S_t, 2)
print(f"S''_nb = {S2_nb} \nS_t = {S_t}")

S_at_1 = np.around((np.sqrt(P_nb[2]**2 + Q2_nb[2]**2) + np.sqrt(P_nb[3]**2 + Q_priv[3]**2) + np.sqrt(P_nb[4]**2 + Q_priv[4]**2)) / (1.2 * (n_t - 1)), 1)
S_at_2 = np.around(S2_nb[2] / (1.2 * 0.5 * (n_t - 1)), 1)
print(f"S_ат_1 = {S_at_1} \nS_ат_2 = {S_at_2}")

S_lrt = np.around(S2_nb[2] / 1.4, 1)
print('S_лрт = ', S_lrt)

S''_nb = [72.9 72.9 31.3 41.5 36.7] 
S_t = [52.1 52.1 29.6 26.2]
S_ат_1 = 92.9 
S_ат_2 = 52.2
S_лрт =  22.4


# Выбор рациональной схемы

In [18]:
#                        K1     K2   13   23
K_0_lep_baz = np.array([1188, 1188, 990, 990])
L_lep = np.array([Lij[0,1], Lij[0,2], Lij[1,3], Lij[2,3]])
k_usl = 1
k_zon = 1
k_def = 4.25 # коэффициент дефляции на 2010 год

K_lep = K_0_lep_baz * k_zon * k_def * k_usl * L_lep
print('Длина линий = ', L_lep)
print('K_ЛЭП = ', K_lep, '\nK_ЛЭП_сумма = ', np.sum(K_lep))

Длина линий =  [41.8 49.2 49.2 41.8]
K_ЛЭП =  [211048.2 248410.8 207009.  175873.5] 
K_ЛЭП_сумма =  842341.5


## Нагрузочные потери в линиях

In [19]:
S = np.around(np.sqrt(P**2 + Q**2), 1)
i = 0
nagr_poter = np.zeros_like(S)
for i in range(len(S)):
    if mask_p1c[i]:
        nagr_poter[i] = np.around((S[i]**2 * R[i]) / 110**2, 2)
    else:
        nagr_poter[i] = np.around((S[i]**2 * R[i]) / 220**2, 2)
        
print(f'S = {S} \nR = {R}')
print('Нагрузочные потери = ', nagr_poter, '\nСуммарные нагрузочные потери = ', np.sum(nagr_poter))

S = [134.9 129.   60.3  54.4  80.2  37.7] 
R = [3.05 3.59 5.81 4.93 1.53 3.16]
Нагрузочные потери =  [1.15 1.23 0.44 0.3  0.81 0.37] 
Суммарные нагрузочные потери =  4.3


## Основные технико-экономические показатели

In [20]:
#                        K1     K2   13   23   34     45
K_0_lep_baz = np.array([1188, 1188, 990, 990, 1403, 1148])
L_lep = np.array([Lij[0,1], Lij[0,2], Lij[1,3], Lij[2,3], Lij[3,4], Lij[4,5]])
k_usl = 1
k_zon = 1
k_def = 4.25 # коэффициент дефляции на 2010 год

K_lep = K_0_lep_baz * k_zon * k_def * k_usl * L_lep
print('Длина линий = ', L_lep)
print('K_ЛЭП = ', K_lep, '\nK_ЛЭП_сумма = ', np.sum(K_lep))

Длина линий =  [41.8 49.2 49.2 41.8 25.9 25.9]
K_ЛЭП =  [211048.2   248410.8   207009.    175873.5   154435.225 126366.1  ] 
K_ЛЭП_сумма =  1123142.825


In [21]:
dP_ud_kor_300 = 0.84
dP_ud_kor_120 = 0.08
F_a = np.array([400, 400, 240, 240, 240, 120])
dP_kor = np.zeros_like(L_kolca)
i = 0
for i in range (len(L_kolca)):
    if mask_p1c[i]:
        dP_kor[i] = np.around(dP_ud_kor_120 * 10**(-3) * 120 / F_a[i] * L_kolca[i] * 2, 5)
    else:
        dP_kor[i] = np.around(dP_ud_kor_300 * 10**(-3) * 300 / F_a[i] * L_kolca[i], 5)
print('dP_кор = ', dP_kor, '\ndP_кор_сумм = ', np.sum(dP_kor))

dP_кор =  [0.02633 0.031   0.05166 0.04389 0.00207 0.00414] 
dP_кор_сумм =  0.15909
